- 了解数据如何收集，数据的格式
- 了解如何用深度学习训练
- 了解深度强化学习中的离线学习（在数据集中学习）是如何进行的
- 了解深度强化学习中的在线学习（直接在游戏中学习）是如何进行的

# 1 collect_data.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/collect_data.py)

### 1.1 get_output()

对按键信息进行独热编码  
注意，这部分的定义非常重要，后面的很多设置都与它相适应

~~~python
def get_output(keys):    # 对按键信息进行独热编码

    output = [0, 0, 0, 0, 0]

    if   'J' in keys:
        output[0] = 1    # 等同于[1, 0, 0, 0, 0]
    elif 'K' in keys:
        output[1] = 1    # 等同于[0, 1, 0, 0, 0]
    elif 'LSHIFT' in keys:
        output[2] = 1    # 等同于[0, 0, 1, 0, 0]
    elif 'SPACE' in keys:
        output[3] = 1    # 等同于[0, 0, 0, 1, 0]
    else:
        output[4] = 1    # 等同于[0, 0, 0, 0, 1]

    return output
~~~

### 1.2 class Data_collection

##### 1.2.1 def \__init__(self, target)

~~~python
self.target = target    # 目标
self.dataset = list()    # 保存数据的容器
self.save_path = os.path.join('The_battle_memory', self.target)    # 保存的位置
self.reward_system = RewardSystem()    # 奖惩系统

self.step = 0    # 计步器
~~~

##### 1.2.2 def save_data(self)

保存数据，统一名称为 f'training_data-{n}.npy'，例如 training_data-123.npy  
从1开始检测文件名是否存在于保存路径，直到检测到不存在就保存

~~~python
n = 1
while True:    # 直到找到保存位置并保存就 break
    filename = f'training_data-{n}.npy'
    save_path = os.path.join(self.save_path, filename)
    if not os.path.exists(save_path):    # 没有重复的文件名就执行保存并退出
        np.save(save_path, self.dataset)
        break
    n += 1
return filename[:-4]
~~~

##### 1.2.3def collect_data(self)

按 'T' 开始，然后等待按 'P' 结束
~~~python
def collect_data(self):

    paused = True
    while True:
        last_time = time.time()
        keys = key_check()
        if paused:
            if 'T' in keys:
                paused = False
        else:

            self.step += 1

            """
            核心部分，看下面
            """

            if 'P' in keys:    # 结束，保存数据
                filename = self.save_data()    # 保存数据，保存结束后返回符合条件的文件名
                self.reward_system.save_reward_curve(
                    save_path = os.path.join('Data_quality', self.target, filename+'.png')
                )    # 绘制 reward 曲线并保存在当前目录
                break
~~~

**核心部分**  
- 记录数据的条件是非黑屏  
- 数据采集降频的目的是缓解数据不平衡和提高 action 与 reward 的对应程度
    - 按键检测的原理是检测当前键盘上正在按着的键，我们的操作有按键有松键，按键对应了4种动作（攻防垫跳），松键只对应1个动作（其他），所以会有大量数据集中在松键状态，而采集频率的增加则会加剧这种情况。
    - 游戏接受玩家的按键信号，然后让人物执行相应的动作，过一会玩家从屏幕观测到状态变化（生命值和架势的变化）。如果采集频率过高，会导致状态变化可能存在好几个新状态之后， action 与 reward 不对应会导致训练出现严重的后果甚至训练结果是无效的。
~~~python
screen = get_screen()    # 获取屏幕图像
if not (np.sum(screen == 0) > 5000):    # 正常情况下不会有那么多值为0的像素点，除非黑屏了
    action = get_output(keys)    # 获取按键输出
    self.dataset.append([screen, action])    # 图像和输出打包在一起，保证一一对应

    status = get_status(screen)
    reward = self.reward_system.get_reward(status)    # 计算 reward

    # 降低数据采集的频率，两次采集的时间间隔为0.1秒
    t = 0.1-(time.time()-last_time)
    if t > 0:
        time.sleep(t)
~~~

### 1.3 数据预览

**注意：请先收集至少一个数据集**

- Data_quality (存放收集数据时记录的reward曲线)
    - Genichiro_Ashina （苇名弦一郎）
        - training_data-1.png （第一个战斗数据的reward曲线）
- 
- The_battle_memory
    - Genichiro_Ashina （苇名弦一郎）
        - training_data-1.npy （第一个战斗数据）

In [1]:
import os

import cv2
import numpy as np
import pandas as pd

target = 'Genichiro_Ashina' # 苇名弦一郎
path = os.path.join('The_battle_memory', target, f'training_data-{1}.npy')
data = np.load(path, allow_pickle=True)
print(f'当前数据集所包含数据量：{data.shape[0]}, 每个数据都有 {data.shape[1]} 个数据，分别是图像数据和按键数据')

当前数据集所包含数据量：1666, 每个数据都有 2 个数据，分别是图像数据和按键数据


In [2]:
screen = data[0][0]    # 选取了第一个数据的图像部分
print('图像', screen)
print('形状', screen.shape)

图像 [[77 85 74 ... 61 63 63]
 [60 36 42 ... 24 25 24]
 [58 53 41 ... 24 25 29]
 ...
 [33 27 28 ... 74 67 71]
 [35 31 30 ... 85 89 89]
 [38 33 32 ... 71 73 73]]
形状 (270, 480)


In [3]:
action_value = data[0][1]    # 选取了第一个数据的按键记录部分
print('按键记录', action_value)

按键记录 [0, 0, 0, 0, 1]


In [4]:
# 统计动作分布
def motionCounts(Y):

    columns=['攻击', '弹反', '垫步', '跳跃', '其他']
    df = pd.DataFrame(Y, columns=columns, dtype=np.uint8)
    
    total = len(df)

    for motion in columns:
        motion_count = len(df)-df[motion].value_counts()[0]
        print(f'motion: {motion:>3}, 记录到的次数: {motion_count:>5}')

Y = np.array([i[1] for i in data])
motionCounts(Y)

motion:  攻击, 记录到的次数:   486
motion:  弹反, 记录到的次数:   130
motion:  垫步, 记录到的次数:    40
motion:  跳跃, 记录到的次数:     4
motion:  其他, 记录到的次数:  1006


In [5]:
print('以视频的形式展示数据')
data = data[-200:-50]    # 展示其中150个
Remaining = len(data)
for screen, action_value in data:
    action = np.argmax(action_value)

    if   action == 0:
        action = '攻击'
    elif action == 1:
        action = '弹反'
    elif action == 2:
        action = '垫步'
    elif action == 3:
        action = '跳跃'
    elif action == 4:
        action = '其他'

    cv2.imshow('screen', screen)
    cv2.waitKey(30)

    Remaining -= 1
    print(f'\r 剩余: {Remaining:>4}, 动作:{action}, {action_value}', end='\n') # end=''
    if cv2.waitKey(1) & 0xFF == ord('q'):    # 按 q 键关闭视频
        cv2.destroyAllWindows()
        break
else:
    cv2.destroyAllWindows()

以视频的形式展示数据
 剩余:  149, 动作:其他, [0, 0, 0, 0, 1]
 剩余:  148, 动作:其他, [0, 0, 0, 0, 1]
 剩余:  147, 动作:其他, [0, 0, 0, 0, 1]
 剩余:  146, 动作:其他, [0, 0, 0, 0, 1]
 剩余:  145, 动作:垫步, [0, 0, 1, 0, 0]
 剩余:  144, 动作:其他, [0, 0, 0, 0, 1]
 剩余:  143, 动作:其他, [0, 0, 0, 0, 1]
 剩余:  142, 动作:其他, [0, 0, 0, 0, 1]
 剩余:  141, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  140, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  139, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  138, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  137, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  136, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  135, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  134, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  133, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  132, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  131, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  130, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  129, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  128, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  127, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  126, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  125, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  124, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  123, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  122, 动作:弹反, [0, 1, 0, 0, 0]
 剩余:  121, 动作:弹反, [0, 1, 0, 0, 0]
 剩余

# 2 model.py

注意：我提供的模型不是最优的，有能力的可以自己重写  
要求：输入形状相关参数（width, height, frame_count），输出相关参数（outputs），需要编译（model.compile）

# 3 train_with_dl.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/train_with_dl.py)

- **target** 指定训练的对象
- 注：必须 start < end
    - **start** 指定第一个训练的数据集
    - **end** 指定最后一个训练的数据集
- **batch_size** 训练时的批大小
- **epochs** 每个数据集训练的次数
    - 如果你搜集的数据足够多，并且是按照上面的要求做的，那完全可以设置成1
    - 因为既然每个数据都是一场完整的战斗，那么真正的epochs其实是数据集的数量，除非你每场战斗是风格都不一样
- **model_weights** 用以支持增量学习。默认为None，从零开始训练。
    - 当你搜集新的数据集之后想继续训练，可以把这个参数设置为要继续训练的模型的路径，这样就可以继续训练了
    - 当你在训练过程中发生了某种意外终止了训练，可以把这个参数设置为要继续训练的模型的路径，然后把发生意外时正在训练的那个数据集的序号设置为start，就可以继续训练了

~~~python
def train(
    target,
    start=1,
    end=1,
    batch_size=8,
    epochs=1,
    model_weights=None
    ):
    
    model = MODEL(ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT,
        outputs = n_action,
        model_weights = model_weights
    )
    model.summary()
    
    """
    核心部分，看下面
    """
~~~

**核心部分**
~~~python
model_weights = 'dl_weights.h5'

# 读取一个数据集训练，然后再读取下一个数据集训练，以此类推
for i in range(start, end+1):

    filename = f'training_data-{i}.npy'
    data_path = os.path.join('The_battle_memory', target, filename)

    if os.path.exists(data_path):    # 确保数据集存在
        """
        训练部分，看下面
        """
    else:
        print(f'{filename} does not exist ')
~~~

**训练部分**
~~~python
# 加载数据集
data = np.load(data_path, allow_pickle=True)

# 数据集处理成预训练格式
X = np.array([roi(i[0], x, x_w, y, y_h) for i in data]).reshape(-1, ROI_WIDTH, ROI_HEIGHT, FRAME_COUNT)
Y = np.array([i[1] for i in data])

# 训练模型，然后保存
model.fit(X, Y, batch_size=batch_size, epochs=epochs, verbose=1)
model.save_weights(model_weights)
~~~

# 4 learn_offline.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/learn_offline.py)

由 train_with_dl.py 升级而来

- **target** 指定训练的对象
- 注：必须 start < end
    - **start** 指定第一个训练的数据集
    - **end** 指定最后一个训练的数据集
- **model_weights** 用以支持增量学习。默认为None。没有指定需要增量学习的模型的路径的话，就从零开始训练。
    - 当你搜集新的数据集之后想继续训练，可以把这个参数设置为要继续训练的模型的路径，这样就可以继续训练了
    - 当你在训练过程中发生了某种意外终止了训练，可以把这个参数设置为要继续训练的模型的路径，然后把发生意外时正在训练的那个数据集的序号设置为start，就可以继续训练了
- **save_path** 指定模型权重保存的路径

~~~python
def learn_offline(
    target,
    start =1,
    end = 1,
    model_weights=None,
    save_path=None
    )
~~~

~~~python
# 依次读取训练集进行离线学习
for i in range(start, end+1):

    filename = f'training_data-{i}.npy'
    data_path = os.path.join('The_battle_memory', target, filename)

    if os.path.exists(data_path):    # 确保数据集存在

        # 加载数据集
        data = np.load(data_path, allow_pickle=True)

        """
        学习过程，看下面
        """
        
        sekiro_agent.save_evaluate_network()    # 这个数据学习完毕，保存网络权重
        sekiro_agent.reward_system.save_reward_curve()    # 绘制 reward 曲线并保存在当前目录

    else:
        print(f'{filename} does not exist ')
~~~

**学习过程**  
主要思路：获取 (状态S、动作A、奖励R 和 新状态S')，存储以供经验回放，然后目标网络和评估网络根据各自设置的更新频率更新自身网络参数  
~~~python
for step in range(len(data)-1):

    # ---------- (S, A, R, S') ----------
    
    # 读取 状态S、动作A 和 新状态S'
    screen = data[step][0]           # 状态S
    action = data[step][1]           # 动作A
    next_screen = data[step+1][0]    # 新状态S'

    # 获取 奖励R
    status = get_status(screen)
    reward = sekiro_agent.reward_system.get_reward(status)    # 奖励R

    # ---------- store ----------
    
    # 集齐 (S, A, R, S')，开始存储
    sekiro_agent.replayer.store(
        roi(screen, x, x_w, y, y_h),
        np.argmax(action),
        reward,
        roi(next_screen, x, x_w, y, y_h)
    )    # 存储经验

    # ---------- train ----------
    
    if step >= sekiro_agent.batch_size:
        if step % update_freq == 0:
            sekiro_agent.learn()    # 更新评估网络
            sekiro_agent.save_evaluate_network()    # 保存网络权重
        if step % target_network_update_freq == 0:    # 更新目标网络
            sekiro_agent.update_target_network()
~~~

# 5 learn_online.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/learn_online.py)

由 collect_data.py 变化而来

~~~python
def learn_online(model_weights=None, save_path=None)
~~~

- **model_weights** 指定模型权重的路径。
- **save_path** 指定模型权重保存的路径。注：设置该参数的同时会开始训练模式

~~~python
if save_path:
    train = True
else:
    train = False
~~~

按 'T' 开始，然后等待按 'P' 结束
~~~python
paused = True

step = 0    # 计步器
step_train = 0    # 计步器

while True:

    last_time = time.time()
    keys = key_check()
    if paused:
        screen = get_screen()    # 首个 状态S，但是在按 'T' 之前，它会不断更新
        if 'T' in keys:
            paused = False
    else:

        step += 1
        
        """
        核心部分，看下面
        """

        # 降低数据采集的频率，两次采集的时间间隔为0.1秒
        t = 0.1-(time.time()-last_time)
        if t > 0:
            time.sleep(t)

        if 'P' in keys:
            if train:
                sekiro_agent.save_evaluate_network()    # 学习完毕，保存网络权重
            sekiro_agent.reward_system.save_reward_curve(save_path='learn_online.png')    # 绘制 reward 曲线并保存在当前目录
            break
~~~

**核心部分**  
- 按 'T' 之后，获得首个状态S，然后依次获得 动作A、奖励R 和 新状态S'，再进入下一个轮回  
- 第二个轮回开始
    1. 原本的 **新状态S'** 变成 **当前状态S**
    2. 由 **当前状态S** 选取 **动作A**
    3. 由 **当前状态S** 获取 **奖励R**
    4. 观测 **新状态S'**
    - 进入下一个轮回
- 这样做的目的是
    1. 集齐 (S, A, R, S')
    2. 保证 状态S 和 新状态S' 连续

~~~python
# ---------- (S, A, R, S') ----------

# 选取动作，同时执行动作
action = sekiro_agent.choose_action(screen, train)    # 动作A

# 获取 奖励R
status = get_status(screen)
reward = sekiro_agent.reward_system.get_reward(status)    # 奖励R

next_screen = get_screen()    # 新状态S'

# ---------- 下一个轮回 ----------

screen = next_screen    # 状态S

"""
训练部分，看下面
"""
~~~

**训练部分**  
~~~python
if train:
    if not (np.sum(screen == 0) > 5000 or np.sum(next_screen == 0) > 5000):    # 正常情况下不会有那么多值为0的像素点，除非黑屏了
        
        # ---------- store ----------
        
        # 集齐 (S, A, R, S')，开始存储
        sekiro_agent.replayer.store(
            roi(screen, x, x_w, y, y_h),
            np.argmax(action),
            reward,
            roi(next_screen, x, x_w, y, y_h)
        )    # 存储经验

        # ---------- train ----------
        
        step_train += 1
        if step_train >= sekiro_agent.batch_size:
            if step_train % update_freq == 0:
                sekiro_agent.learn()    # 更新评估网络
                sekiro_agent.save_evaluate_network()    # 保存网络权重
            if step_train % target_network_update_freq == 0:    # 更新目标网络
                sekiro_agent.update_target_network()
~~~